08 より進んだトークン分割、語幹処理、見出し語化
=======================================

* `CountVectorizer`や`TfidVectorizer`の行う特徴量抽出は比較的単純で、まだまだ向上の余地がある

    * より洗練されたテキスト処理アプリケーションでよく改良されているのは、`BoW`モデル構築の最初の過程(`トークン分割`)
    
    * このステップでは、特徴量抽出に用いられる単語の構成を定める

* ボキャブラリの中には、複数形、動詞の変化などが含まれる

    * この問題は、個々の単語を`語幹`を使って表現すれば解決する
    
    * これは同じ語幹を持つ全ての単語を特定する(`融合`)する必要がある
    
    * これを、単語の末尾につく特定の形を取り除くといったようなルールベースのヒューリスティクスで行う場合、`語幹処理`と呼ぶ
    
    * 知られている単語に対して辞書を用いて、単語の文章での役割を考慮して行う場合には`見出し語化`とよび、単語の標準的な形を`見出し語`と呼ぶ
    
    * 語幹処理、見出し語化はいずれも単語の正規形を取り出そうと試みる`正規化`と呼ぶ

* 正規化を理解するために、`語幹処理`と`見出し語化`を比較してみる

    * `語幹処理`には広く用いられているヒューリスティクスの集合であるPorter stemmer(nltkパッケージ)を用いる
    
    * `見出し語化`にはspacyパッケージのものを用いる

In [4]:
import spacy
import nltk

# spacyの英語モデルをロード
en_nlp = spacy.load('en')
# nltkのPorter stemmerのインスタンスを作成
stemmer = nltk.stem.PorterStemmer()

# spacyによる見出し語化とnltkによる語幹処理を比較する関数を定義
def compare_normalization(doc):
    # spacyで文書をトークン分割
    doc_spacy = en_nlp(doc)
    # spacyで見つけた見出し語を表示
    print("Lemmatization:")
    print([token.lemma_ for token in doc_spacy])
    # Porter stemmerで見つけたトークンを表示
    print("Stemming:")
    print([stemmer.stem(token.norm_.lower()) for token in doc_spacy])

* 見出し語化と語幹処理の違いが分かるように作り込んだ文章を使って比較してみる

In [5]:
compare_normalization(u"Our meeting today was worse than yesterday, "
                       "I'm scared of meeting the clients tomorrow.")

Lemmatization:
['-PRON-', 'meeting', 'today', 'be', 'bad', 'than', 'yesterday', ',', '-PRON-', 'be', 'scared', 'of', 'meet', 'the', 'client', 'tomorrow', '.']
Stemming:
['our', 'meet', 'today', 'wa', 'wors', 'than', 'yesterday', ',', 'i', 'am', 'scare', 'of', 'meet', 'the', 'client', 'tomorrow', '.']


* 語幹処理は、単語を切り縮めて語幹にすることしかないので、"was"は"wa"となってしまう

* 一方、見出し語化では正しい動詞の基本形の"be"となっている

    * 同様に、見出し語化では"worse"を正しく"bad"と正規化できているが、語幹処理では"wors"となっている
    
* もう一つ大きな違いがある

    * 語幹処理では2回出現する"meeting"をどちらも"meet"にしてしまっている
    
    * 見出し語化では最初の"meeting"は名詞と判断されてそのまま残る
    
    * 2度目の"meeting"は動詞として"meet"とされている
    
* 一般に見出し語化は語幹処理よりも複雑な処理で、機械学習におけるトークンの正規化に用いるとより良い結果が得られる
    

* scikit-learnはどちらの正規化手法も実装されていないが、`CountVectorizer`の`tokenizer`パラメータで、文書をトークン分割器を指定することができる

    * spacyの見出し語化機能を使って、文字列から見出し語の列を作る関数を作って指定すれば良い

In [7]:
from sklearn.feature_extraction.text import CountVectorizer
# 技術的詳細：CountVectorizerが用いている正規表現ベースの
# トークン分割器を用いて、見出し語化だけにspacyを用いるのが望ましい
# このため、en_nlp.tokenizer(spacyのトークン分割器)を、正規表現ベースのトークン分割器に置き換えている
import re
# CountVectorizerで用いられているトークン分割用の正規表現
regexp = re.compile('(?u)\\b\\w\\w+\\b')
# spacyの言語モデルを読み込み、トークン分割器を取り出す
en_nlp = spacy.load('en', disable=['parser', 'ner'])
old_tokenizer = en_nlp.tokenizer
# トークン分割器を先ほどの正規表現で置き換える
en_nlp.tokenizer = lambda string: old_tokenizer.tokens_from_list(
    regexp.findall(string))

# spacyの文書処理パイプラインを用いてカスタムトークン分割器を作る(正規表現を用いたトークン分割器を組み込んである)
def custom_tokenizer(document):
    doc_spacy = en_nlp(document)
    return [token.lemma_ for token in doc_spacy]

# COuntVectorizerをカスタムトークン分割器を使って定義する
lemma_vect = CountVectorizer(tokenizer=custom_tokenizer, min_df=5)

* データを変換してボキャブラリのサイズを見てみる

In [9]:
from sklearn.datasets import load_files

reviews_train = load_files("/Users/MacUser/data/aclImdb/train/")
text_train, y_train = reviews_train.data, reviews_train.target
text_train = [doc.replace(b"<br />", b" ") for doc in text_train]
reviews_test = load_files("/Users/MacUser/data/aclImdb/test/")
text_test, y_test = reviews_test.data, reviews_test.target
text_test = [doc.replace(b"<br />", b" ") for doc in text_test]

from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer().fit(text_train)
X_train = vect.transform(text_train)

import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
scores = cross_val_score(LogisticRegression(), X_train, y_train, cv=5)

from sklearn.model_selection import GridSearchCV
param_grid = {'C': [0.001, 0.01, 0.1, 1, 10]}
grid = GridSearchCV(LogisticRegression(), param_grid, cv=5)
grid.fit(X_train, y_train)

X_test = vect.transform(text_test)

vect = CountVectorizer(min_df=5).fit(text_train)
X_train = vect.transform(text_train)
grid = GridSearchCV(LogisticRegression(), param_grid, cv=5)
grid.fit(X_train, y_train)

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import make_pipeline
pipe = make_pipeline(TfidfVectorizer(min_df=5, norm=None),
                     LogisticRegression())
param_grid = {'logisticregression__C': [0.001, 0.01, 0.1, 1, 10]}

grid = GridSearchCV(pipe, param_grid, cv=5)
grid.fit(text_train, y_train)

vectorizer = grid.best_estimator_.named_steps["tfidfvectorizer"]
# 訓練データセットを変換
X_train = vectorizer.transform(text_train)
# それぞれの特徴量のデータセット中での最大値を見つける
max_value = X_train.max(axis=0).toarray().ravel()
sorted_by_tfidf = max_value.argsort()
# 特徴量名を取得
feature_names = np.array(vectorizer.get_feature_names())
sorted_by_idf = np.argsort(vectorizer.idf_)

/Users/MacUser/anaconda2/envs/tf140/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/MacUser/anaconda2/envs/tf140/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/MacUser/anaconda2/envs/tf140/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/MacUser/anaconda2/envs/tf140/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [12]:
# 見出し語化を行うCountVectorizerでtext_trainを変換
X_train_lemma = lemma_vect.fit_transform(text_train)
print("X_train_lemma.shape: {}".format(X_train_lemma.shape))

# 比較のために標準のCountVectorizerでも変換
vect = CountVectorizer(min_df=5).fit(text_train)
X_train = vect.transform(text_train)
print("X_train.shape: {}".format(X_train.shape))

X_train_lemma.shape: (25000, 21575)
X_train.shape: (25000, 27271)


* この結果から分かるように、見出し語化によって特徴量が27,271(標準のCountVectrizer処理の結果)から、21,596へと減少している

* 見出し語化は、特定の特徴量を融合するもので、ある種の正則化と見なすことができる

    * したがって、見出し語化によって最も性能が向上するのはデータセットが小さい場合であることが予測できる
    
* 見出し語化の有効性を確認するために、`StratifiedShuffleSplit`を用い、データの1%だけを訓練データとし、残りをテストデータとして交差検証を行う

In [13]:
# データの1%だけを訓練セットとして用いてグリッドサーチを行う
from sklearn.model_selection import StratifiedShuffleSplit

param_grid = {'C': [0.001, 0.01, 0.1, 1, 10]}
cv = StratifiedShuffleSplit(n_splits=5, test_size=0.99,
                            train_size=0.01, random_state=0)
grid = GridSearchCV(LogisticRegression(), param_grid, cv=cv)
# 標準のCountVectorizerを用いてグリッドサーチを実行
grid.fit(X_train, y_train)
print("Best cross-validation score "
      "(standard CountVectorizer): {:.3f}".format(grid.best_score_))
# 見出し語化付きで、グリッドサーチを実行
grid.fit(X_train_lemma, y_train)
print("Best cross-validation score "
      "(lemmatization): {:.3f}".format(grid.best_score_))

/Users/MacUser/anaconda2/envs/tf140/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Best cross-validation score (standard CountVectorizer): 0.721


/Users/MacUser/anaconda2/envs/tf140/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Best cross-validation score (lemmatization): 0.735


* この場合、見出し語化を行っても性能は若干向上する程度

    * 他の特徴量抽出技術と同様に、結果はデータセットによって異なる
    
    * 見出し語化や語幹処理によって、より良いモデルを作る役に立つ場合もある
    
    * 特定のタスクに対して性能の最後のひとしずくまで絞り出したい際にはこの技術を使ってみることをオススメする

| 版   | 年/月/日   |
| ---- | ---------- |
| 初版 | 2019/04/06 |